In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from utils.data import CatsAndDogs
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
from models.classifier import ClassifierForMNIST

In [2]:
BATCH_SIZE = 16
ETA = 1e-4
EPOCHS = 10

In [3]:
def compute_accuracy(ps, labels):
    topk_vals, topk_indices = ps.topk(k=1, dim=1)
    equality = (topk_indices.squeeze() == labels).float()
    acc = equality.mean()
    return acc

In [4]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    N = x.size(0)
    x = x.view(N, 1, 28, 28)
    
    logps = model(x)
    loss = F.nll_loss(logps, y.long()) # nn.NLLLoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    ps = torch.exp(logps)
    acc = compute_accuracy(ps, y.long())
    
    return loss.item(), acc.item()

In [5]:
def eval_step(context, x, y):
    model = context["model"]
    
    N = x.size(0)
    x = x.view(N, 1, 28, 28)
    
    logps = model(x)
    loss = F.nll_loss(logps, y.long()) # nn.NLLLoss()(ps, y) 와 같음
    
    ps = torch.exp(logps)
    acc = compute_accuracy(ps, y.long())
    
    return loss.item(), acc.item()

In [6]:
def train():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (1.0,))
    ])
    
    trainset = MNIST("../data", transform=transform, download=False, train=True)
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = MNIST("../data", transform=transform, download=False, train=False)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    model = ClassifierForMNIST().cuda()
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [7]:
train()

Epochs 1/10
Train loss: 0.16323918, train acc: 0.9503
Test loss: 0.06671247, test acc: 0.9791
Epochs 2/10
Train loss: 0.05828028, train acc: 0.9819
Test loss: 0.06355739, test acc: 0.9789
Epochs 3/10
Train loss: 0.04144047, train acc: 0.9869
Test loss: 0.04513464, test acc: 0.9868
Epochs 4/10
Train loss: 0.03090168, train acc: 0.9904
Test loss: 0.04770483, test acc: 0.9851
Epochs 5/10
Train loss: 0.02363882, train acc: 0.9924
Test loss: 0.05327287, test acc: 0.9837
Epochs 6/10
Train loss: 0.01803125, train acc: 0.9944
Test loss: 0.05035236, test acc: 0.9862
Epochs 7/10
Train loss: 0.01448422, train acc: 0.9953
Test loss: 0.04739158, test acc: 0.9867
Epochs 8/10
Train loss: 0.01147383, train acc: 0.9962
Test loss: 0.05514707, test acc: 0.9868
Epochs 9/10
Train loss: 0.00980160, train acc: 0.9966
Test loss: 0.06190576, test acc: 0.9859
Epochs 10/10
Train loss: 0.00768372, train acc: 0.9974
Test loss: 0.07446346, test acc: 0.9844
